<a href="https://colab.research.google.com/github/N34R20/DeepLearningHumai/blob/main/vivo_clase_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            self._modules[str(idx)] = module

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [3]:
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

In [4]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # Random weight parameters for which gradients will not be computed and,
        # thus, kept constant during training
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # Use the constant parameters created, as well as `relu` and `mm` functions
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # Reuses the fully connected layer. This is equivalent to two fully connected
        # layers sharing parameters
        X = self.linear(X)
        # Control flow
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()


In [5]:
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

In [6]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.zeros_(m.bias)

1. Crea una nueva clase denominada MyCustomNet que herede de MySequential. Esta nueva red debe incorporar al menos dos capas lineales definidas por la clase MyLinear y una capa de normalización centrada (CenteredLayer). Los parámetros de las capas lineales quedan a tu discreción, pero ten en cuenta la compatibilidad de las dimensiones.

In [80]:
# 1. Crear una nueva clase de red
class MyCustomNet(MySequential):
  def __init__(self):
    super().__init__(MyLinear(10, 20),
    CenteredLayer(),
    MyLinear(20, 5))



2. Inicializa la red MyCustomNet y aplica la inicialización normal. Luego, imprime los pesos y bias de la primera capa lineal del modelo.

In [81]:
for x in net.state_dict():
  print(net.state_dict()[x].shape)

torch.Size([10, 20])
torch.Size([20])
torch.Size([20, 5])
torch.Size([5])


In [84]:
# 2. Inicializar y aplicar la inicialización normal. Imprimir los pesos

net = MyCustomNet()
net.apply(init_normal)

print("pesos primer capa \n", net._modules['0'].weight.data, "\n")
print("bias primer capa \n",  net._modules['0'].bias.data, "\n")
print("pesos tercer capa \n", net._modules['2'].weight.data, "\n")
print("bias tercer capa \n", net._modules['2'].bias.data, "\n")

pesos primer capa 
 tensor([[ 1.7616, -0.0072, -1.5197,  0.7433, -1.8664, -0.4593,  0.2137,  0.9204,
         -0.7881, -0.4063,  1.2151, -1.1631, -0.2710, -1.7196, -0.8544,  0.3439,
         -1.5644, -0.8086,  0.1395, -0.3382],
        [-0.0293,  0.9719, -0.5568, -0.1523,  0.3836,  0.2594,  0.5429, -0.0175,
         -0.5591, -0.1506, -0.5539, -1.4990, -0.7488,  0.0085, -1.4280,  2.0146,
          0.8386,  0.6251, -0.5426,  0.4333],
        [-0.3225, -0.6956, -0.9639, -0.8935, -0.2561,  0.3474,  1.1949, -1.7606,
         -1.4442, -1.7526, -0.2415,  0.2798, -0.7187,  0.0372, -0.4341,  1.5997,
         -0.1551, -0.7451,  0.3334,  0.3546],
        [ 0.6258, -0.1696,  0.5785, -1.0781,  2.0216, -1.3460, -1.5602, -0.3701,
         -0.2821,  0.4759, -0.8959,  2.6348, -0.8305, -0.4017,  1.1588, -1.4111,
         -0.0907, -0.0321,  0.5335, -0.7593],
        [ 1.4215, -1.7267,  0.6872,  0.1680, -2.0252, -0.5191,  1.3700,  1.4379,
          0.8016, -0.7865, -1.5478,  1.3349, -0.6247, -1.1626,  1.9

3. Guarda los parámetros del modelo en un archivo.

In [61]:
# 3. Guardar los parámetros del modelo
torch.save(net.state_dict(), 'net.params')

4. Carga los parámetros desde el archivo a una nueva instancia de MyCustomNet.

In [63]:
# 4. Cargar los parámetros en una nueva instancia
clone = MyCustomNet()
clone.load_state_dict(torch.load('net.params'))

<All keys matched successfully>

5. Evalúa el modelo cloned_net utilizando un tensor de entrada ficticio de dimensiones adecuadas. Recuerda llamar al método eval antes de pasar cualquier dato a través del modelo para asegurarte de que la red está en modo de evaluación.

In [64]:
# 5. Evaluar el modelo clonado
clone.eval()

MyCustomNet(
  (hidden1): MyLinear()
  (hidden2): CenteredLayer()
  (output): MyLinear()
)

6. Imprime los pesos y bias de la primera capa lineal del modelo clonado. Son iguales a los de la red original?

In [68]:
# 6. Imprimir los pesos y bias de la primera capa lineal

print("pesos primer capa \n", net._modules['hidden1'].weight.data, "\n")
print("bias primer capa \n",  net._modules['hidden1'].bias.data, "\n")

pesos primer capa 
 tensor([[-1.3080e+00, -1.0253e+00,  1.4961e-01, -7.1866e-01,  1.3914e+00,
          1.0232e+00,  4.4261e-01,  5.0584e-01, -1.6189e+00,  2.2112e-01,
          2.3416e-02, -9.6936e-01,  2.9629e-01,  1.6683e-01, -4.3601e-01,
         -1.3189e+00, -3.1725e-01,  3.9147e-01, -2.0089e+00, -7.8559e-03],
        [ 1.4731e+00,  1.2827e+00,  4.4902e-01,  1.3625e-01,  1.1962e+00,
          8.3547e-01, -9.6020e-02, -1.1153e+00,  1.4132e+00, -8.1999e-01,
         -2.9014e-01,  2.8460e-01, -1.8180e-01,  7.2537e-01, -3.8369e-01,
         -4.3370e-02, -2.4077e+00, -5.1402e-01,  1.4732e+00, -2.0842e+00],
        [ 5.4577e-01, -1.0409e+00, -4.8822e-01,  2.3097e+00,  8.8667e-01,
          3.5133e-02, -8.9454e-01, -2.6321e+00, -9.2030e-01,  1.6322e+00,
          3.6203e-02,  5.1434e-01,  6.7752e-01,  1.7216e+00, -2.4655e-03,
         -8.1402e-01, -5.4561e-01,  3.5320e-01,  5.4876e-01, -1.2570e-01],
        [ 1.6277e+00,  1.0455e+00,  7.7089e-01, -8.1825e-01,  1.0809e+00,
         -1.130